## Prep Data for analysis

This is a Jupyter notebook.

To run all cells in the notebook use `Cell --> Run All`.

To run cells one at a time click into the first code cell and key `Shift-Enter` in each cell in sequence.

More information on Jupyter notebooks can be found
[here](http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Running%20Code.html).

In [1]:
## Set the plot window sizes within the notebook (in inches)
options(repr.plot.width=6, repr.plot.height=8)

#   Script parameters setup, setting reasonable defaults

In [2]:
opt <- list()
opt$outdir    <- './user_data/output'
opt$map.human <- FALSE # Set to TRUE if you want to regenerate the human mapping file.
opt$datadir   <- './user_data' # Directory containing the input data

In [3]:
## Load libraries
library(knitr)
library(plyr)
library(dplyr)
library(broom)
library(biobroom)
library(tidyr)
library(qvalue)
library(edgeR)
library(biomaRt,pos = "package:base")
library(reshape2)

library(devtools)
knitr::opts_chunk$set(warning = FALSE,
                      message = FALSE,
                      prompt = FALSE)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: limma

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths



### Load the data

In [4]:
## Load the phenotype data 
print('Loading phenotype data')
samplesCanine <- read.csv(paste(opt$datadir,'samples_canine_updated.csv',sep='/'))

[1] "Loading phenotype data"


In [5]:
## Create the count files convert to gene level counts
print('Creating the count files'); flush.console()
fn.txt <- list.files(path=paste(opt$datadir,"dexseq_count/",sep='/'), pattern="*.txt", full.names=FALSE, recursive=FALSE)
fn.txt <- paste(opt$datadir, 'dexseq_count', fn.txt, sep='/')
for(fn in fn.txt) {
  print( paste('Processing sample', fn) ); flush.console()
  dat.fn <- read.table(fn, sep='\t', header=FALSE, stringsAsFactors=FALSE)
  ids.genes <- sapply(dat.fn$V1, function(x) {unlist(strsplit(x,':'))[1] } )

  dat    <- aggregate(dat.fn[,2,drop=FALSE], by=list(ids.genes), FUN=sum)
  rownames(dat) <- dat$Group.1
  dat <- dat[,-1,drop=FALSE]
  dat <- dat[unique(ids.genes),,drop=FALSE]

  write.table( dat, file=paste(opt$datadir,'dexseq_count',paste(tools::file_path_sans_ext(basename(fn)), 'count',sep='.'),sep='/'), sep ='\t', quote=FALSE, col.names=FALSE, row.names=TRUE ) # TODO
}

[1] "Creating the count files"
[1] "Processing sample ./user_data/dexseq_count/10A.txt"
[1] "Processing sample ./user_data/dexseq_count/10B.txt"
[1] "Processing sample ./user_data/dexseq_count/10C.txt"
[1] "Processing sample ./user_data/dexseq_count/10D.txt"
[1] "Processing sample ./user_data/dexseq_count/10F.txt"
[1] "Processing sample ./user_data/dexseq_count/11A.txt"
[1] "Processing sample ./user_data/dexseq_count/11B.txt"
[1] "Processing sample ./user_data/dexseq_count/11C.txt"
[1] "Processing sample ./user_data/dexseq_count/11E.txt"
[1] "Processing sample ./user_data/dexseq_count/11F.txt"
[1] "Processing sample ./user_data/dexseq_count/11G.txt"
[1] "Processing sample ./user_data/dexseq_count/11H.txt"
[1] "Processing sample ./user_data/dexseq_count/11I.txt"
[1] "Processing sample ./user_data/dexseq_count/11J.txt"
[1] "Processing sample ./user_data/dexseq_count/12A.txt"
[1] "Processing sample ./user_data/dexseq_count/12B.txt"
[1] "Processing sample ./user_data/dexseq_count/12C.txt"


In [6]:
## Read in Count Data
print('Loading count data')
counts <- readDGE(samplesCanine$File, paste(opt$datadir,"dexseq_count/",sep='/'), header = FALSE)
minCount <- round((nrow(samplesCanine)/length(levels(samplesCanine$Hist))), 0)
cpms <- cpm(counts)
noint <- rownames(counts) %in% c("_ambiguous_readpair_position","_ambiguous","_empty","_lowaqual","_notaligned")
counts_filtered <- counts[rowSums(cpms > 1) >= minCount & !noint,]
samplesCanine$Hist <- relevel(samplesCanine$Hist,"N")
samplesCanine <- samplesCanine %>% mutate(Patient = factor(PatientNumber))

[1] "Loading count data"


Meta tags detected: _ambiguous, _ambiguous_readpair_position, _empty, _lowaqual, _notaligned


### Prepare Data & Identify Differentially Expressed Genes

In [7]:
## Estimate Dispersion
print('Estimating dispersion')
design <- model.matrix( ~Patient+Hist, samplesCanine)
d <- DGEList(counts = counts_filtered, group = samplesCanine$Hist)
d <- calcNormFactors(d)
d <- estimateGLMTrendedDisp(d, design)
d <- estimateGLMTagwiseDisp(d, design)

[1] "Estimating dispersion"


In [8]:
## Perform DE Analysis
print('Performing differential expresion analysis')
d_fit <- glmFit(d,design)
lrt_list <- list(
    b_n = glmLRT(d_fit,coef = "HistB"),
    m_n = glmLRT(d_fit,coef = "HistM"),
    m_b = glmLRT(d_fit,contrast = makeContrasts(HistM-HistB, levels = d_fit$design)),
    n_b_m = glmLRT(d_fit,coef = c("HistM", "HistB"))
    )
tidy.DGELRT <- function(x, ...) {
  ret <- fix_data_frame(x$table, newcol = "gene")
}
LRTtidied <- ldply(lrt_list, tidy,.id = "contrast") %>%
  group_by(contrast) %>%
  mutate(qval = qvalue(PValue)$qvalues)

[1] "Performing differential expresion analysis"


Warning message in makeContrasts(HistM - HistB, levels = d_fit$design):
“Renaming (Intercept) to Intercept”

### Load mappings to human (if option is set)

In [9]:
## NOTE: Sometimes the bioMart datasets won't load. Just wait a bit and try again, they usually come back up quickly
if(!opt$map.human) {
  print('Using previously generated human mapping file')
  load(paste(opt$datadir,"humanmapping.rda",sep='/'))
} else {
  print('Generating new human mapping file')
  allgenes <- LRTtidied %>% ungroup %>% select(gene) %>% distinct %>% .$gene

  Map_CanEns2Info <-  getBM(attributes = c('ensembl_gene_id','external_gene_name','description'),
                          filters = list(ensembl_gene_id = allgenes),
                          mart = useMart("ensembl",dataset = "cfamiliaris_gene_ensembl"))

  Map_CanEns2HumEns <- getBM(attributes = c('ensembl_gene_id','hsapiens_homolog_ensembl_gene'),
                           filters = list(ensembl_gene_id = allgenes, with_hsapiens_homolog = TRUE),
                           mart = useMart("ensembl",dataset = "cfamiliaris_gene_ensembl"))

  Map_HumEns2Entrez <- getBM(attributes = c('ensembl_gene_id','entrezgene'),
                           filters = 'ensembl_gene_id',
                           values = Map_CanEns2HumEns$hsapiens_homolog_ensembl_gene,
                           mart = useMart("ensembl",dataset = "hsapiens_gene_ensembl"))

  Map_HumEns2Symb <- getBM(attributes = c('ensembl_gene_id','external_gene_name'),
                         filters = 'ensembl_gene_id',
                         values = Map_CanEns2HumEns$hsapiens_homolog_ensembl_gene,
                         mart = useMart("ensembl",dataset = "hsapiens_gene_ensembl"))

  Map_CanEns2HumEnt <- inner_join(Map_CanEns2HumEns,
                                Map_HumEns2Entrez,
                                by = c("hsapiens_homolog_ensembl_gene" = "ensembl_gene_id"))

  Map_CanEns2HumEnt_unique <- Map_CanEns2HumEnt %>%
    select(Can_Ens = ensembl_gene_id, Hum_Ent = entrezgene) %>% distinct %>%
    group_by(Can_Ens) %>%
    filter(length(Can_Ens) == 1) %>%
    group_by(Hum_Ent) %>%
    filter(length(Hum_Ent) == 1) %>%
    ungroup %>% mutate(Hum_Ent = as.character(Hum_Ent))

  Map_CanEns2HumSymb <- inner_join(Map_CanEns2HumEns,
                                 Map_HumEns2Symb,
                                 by = c("hsapiens_homolog_ensembl_gene" = "ensembl_gene_id"))

  Map_CanEns2HumSymb_unique <- Map_CanEns2HumSymb %>%
    select(Can_Ens = ensembl_gene_id,Hum_Symb = external_gene_name) %>% distinct %>%
    group_by(Can_Ens) %>%
    filter(length(Can_Ens) == 1) %>%
    group_by(Hum_Symb) %>%
    filter(length(Hum_Symb) == 1)

  save(Map_CanEns2HumEns,
       Map_CanEns2Info,
       Map_CanEns2HumEnt,
       Map_CanEns2HumEnt_unique,
       Map_CanEns2HumSymb,
       Map_CanEns2HumSymb_unique,file=paste(opt$datadir,"humanmapping.rda",sep='/'))

       #Map_CanEns2HumSymb_unique,file="humanmapping.rda")

} # End human mapping

[1] "Using previously generated human mapping file"


### Generate the profile metrics (later used to generate PEPs)

In [10]:
print('Generating profile metrics')
profileMetrics <- LRTtidied %>%
    filter(contrast!="n_b_m") %>%
 select(contrast, gene, logFC, PValue, qval) %>%
   gather(type, value, -contrast, -gene) %>%
   unite(contrast_type, contrast, type, sep=".") %>%
   spread(contrast_type, value) %>%
inner_join(Map_CanEns2HumSymb_unique,by=c("gene"="Can_Ens"))

[1] "Generating profile metrics"


In [11]:
getCPMWithoutPatient <- function(DGEFit) {
  cpm = cpm(DGEFit$counts,normalized.lib.sizes = TRUE, log = FALSE)
  cpm = cpm + .25 #Add prior
  logcpm = log(cpm)
  patient.cols = grep("Patient", colnames(DGEFit$coefficients))
  due.to.patient = DGEFit$coefficients[, patient.cols] %*% t(DGEFit$design[, patient.cols])
  without.patient = logcpm - due.to.patient
  without.patient = log(exp(without.patient), base = 2) #To make comparable to cpm()
  return(without.patient)
}

### Generate and save the canine expression data matrices

In [12]:
print('Generating canine data matrices')
CPMmatrices <- list(
  raw = cpm(d_fit$counts, normalized.lib.sizes = FALSE, log = FALSE),
  norm = cpm(d_fit$counts, normalized.lib.sizes = TRUE, log = FALSE),
  znorm = t(scale(t(cpm(d_fit$counts, normalized.lib.sizes = TRUE, log = FALSE)),center = TRUE, scale = TRUE)),
  patregressed = getCPMWithoutPatient(d_fit),
  zpatregressed = t(scale(t(getCPMWithoutPatient(d_fit)),center = TRUE,scale = TRUE))
  )

[1] "Generating canine data matrices"


In [13]:
## Save updated files
print('Saving rda files')
save(CPMmatrices, file=paste(opt$outdir,'CPMmatrices.rda',sep='/'))
save(LRTtidied, file=paste(opt$outdir,'LRTtidied.rda',sep='/'))
save(profileMetrics, file=paste(opt$outdir,'profileMetrics.rda',sep='/'))

[1] "Saving rda files"


In [14]:
## Save the dog matrices
print('Saving delimited files')
dat.dog <- CPMmatrices$zpatregressed
names.dog <- intersect( Map_CanEns2HumSymb_unique$Can_Ens, rownames(dat.dog) ) # Only care about genes that map to human
Map_CanEns2HumSymb_unique <- Map_CanEns2HumSymb_unique[ Map_CanEns2HumSymb_unique$Can_Ens %in% names.dog, ]
rownames( Map_CanEns2HumSymb_unique ) <- Map_CanEns2HumSymb_unique$Can_Ens

## Map to human, make dog ID's more dog-obvious
dat.dog <- dat.dog[names.dog,]
rownames(dat.dog) <- Map_CanEns2HumSymb_unique$Hum_Symb

## Save to file
write.table(t(dat.dog), file=paste(opt$outdir,'Canine_RNASeq.tab',sep='/'), sep='\t', col.names=TRUE, row.names=TRUE, quote=FALSE)
write.table(t(dat.dog), file=paste(opt$outdir,'Canine_RNASeq.csv',sep='/'), sep=',', col.names=TRUE, row.names=TRUE, quote=FALSE)

## Save the canine samples data
dog.labs <- data.frame( samplesCanine$Qlabel, samplesCanine$Hist )
colnames(dog.labs) <- c('ID','Hist')
write.table(dog.labs, file=paste(opt$outdir,'Canine_Labels.tab',sep='/'), sep='\t', col.names=TRUE, row.names=FALSE, quote=FALSE)

## Now grab malignant vs normal - remove benign samples (for tumor-normal analyses)
x <- t(dat.dog[ ,as.character(dog.labs[ dog.labs$Hist !='B', 'ID' ]),drop=FALSE])
write.table(x, file=paste(opt$outdir,'Canine_RNASeq_TumorVSNormal.t.tab',sep='/'), sep='\t', col.names=TRUE, row.names=TRUE, quote=FALSE)

print('Finished. Success!')

[1] "Saving delimited files"


Warning message:
“Setting row names on a tibble is deprecated.”

[1] "Finished. Success!"
